In [ ]:
# This is only to fix autocomplete in my version of Jupyter notebook.  Disregard if you don't have this problem
%config Completer.use_jedi = False

# This is to allow the use pyqtgraph for plots
%gui qt

import qutip
from VV0_params import PL1,PL2
import numpy as np
import pyqtgraph as pg
from easy_wave import *
from wave_library import IQ_MW_Pulse,Rect, Empty, Zero
from wave_sim import *


# Qutip Simulation from Waveform

WARNING: Simulation is still experimental.  Doing it with the Simple_Sim object should work, but it can be slow

In [ ]:
# First let's give names to our channels
chs = {
    'sg1_i'  : Channel.ch1_a,
    'sg1_g'  : Channel.ch1_m1,
    
    'sg1_q'  : Channel.ch2_a,
}
sg1_chs = [chs[x] for x in ['sg1_i','sg1_q', 'sg1_g']]

# Creating some basic pulses
mw = IQ_MW_Pulse(300e-9, *sg1_chs, freq_shift=0)
delay = Zero(t=1e-6, ch=chs['sg1_g'])

#Create the sequence
wave = mw + delay + mw + delay + mw + delay + mw
wave &= Rect(t=wave.t, ch = Channel.ch2_m1)

Bz_val = 10
gamma_e=2.8e6
f_plus = PL2['D']#+Bz_val*gamma_e

In [ ]:
%%time
options = qutip.solver.Options(max_step=0.1e-9)

Bx = SG_IQ(f_plus, chs['sg1_i'], chs['sg1_q'], wave, experiment_rate=1e9, axis='x')
Bz = Static(val=Bz_val)
H = Single_VV0_Spin(PL2['D'])
H.add_signal(Bx)
# H.add_signal(Bz)

sim = Simple_Sim(H, Bx.exp_ts, options=options)

plt = sim.plot(Bx.exp_ts)
H.plot_signal('Bx', Bx.exp_ts)#, plt=plt)

In [ ]:
sim.plot(Bx.exp_ts, method='real')

In [ ]:
# Not working
class Simple_Sim():
    def __init__(self, H, ts, psi0=qutip.basis(3,1), options=qutip.solver.Options(), force_recompute=False):
        self.H = H
#         print(H.signals)
#         print(H.get_qutip_descriptor(time_offset=min(ts)))
        if H.is_constant() or H.T is None:
            print("simulating using sesolve")
            self.output = qutip.sesolve(H.get_qutip_descriptor(time_offset=min(ts)), psi0, ts-min(ts), options=options)
        else:
            #Periodic hamiltonian, use Floquet Formalism
            print("simulating using fsesolve")
            self.output = qutip.fsesolve(H.get_qutip_descriptor(time_offset=min(ts)), psi0, ts-min(ts), e_ops=[], T=H.T, args={})

    def plot(self, ts, method='abs', plt=None):
        states = np.array([s.full() for s in self.output.states])[:,:,0]
        if method == 'abs':
            states = np.abs(states)**2
        elif method == 'real':
            states = np.real(states)
        elif method == 'imag':
            states = np.imag(states)

        cs = ['b', 'g', 'r', 'c', 'y', 'w','m']
        if plt is None:
            plt = pg.plot()
        if not self.H.labels is None:
            plt.addLegend()
            for i, ys in enumerate(states.T):
                plt.plot(ts, ys, pen=cs[i%len(cs)], name=self.H.labels[i])
        else:
            for i, ys in enumerate(states.T):
                plt.plot(ts, ys, pen=cs[i%len(cs)])
        return plt


class Sequencer(object):
    def __init__(self, wave, relevent_chs):
        self.seq = self.serialize(wave, relevent_chs)
        self.compress_zeros()
        self.re_id_zeros()

    def serialize(self, wave, relevent_chs):
        if not wave.PULSE_TYPE is None:
            return [wave]

        if type(wave) == AND_Waveform:
            is_relevent = [any([ch in sub_wave.chs for ch in relevent_chs])for sub_wave in wave.wave_list]
            no_relevent_waves = sum(is_relevent)
            if no_relevent_waves == 0:
                return []
            elif no_relevent_waves == 1:
                relevent_wave = wave.wave_list[is_relevent.index(True)]
                return self.serialize(relevent_wave, relevent_chs=relevent_chs)
            else:
                return [wave]

        elif type(wave) == Waveform:
            out_list = []
            is_relevent = [any([ch in sub_wave.chs for ch in relevent_chs])for sub_wave in wave.wave_list]
            for i, sub_wave in enumerate(wave.wave_list):
                if not is_relevent[i]:
                    out_list.append(wave_lib.Zero(t=sub_wave.t, ch=Channel.no_ch))
                else:
                    out_list.extend(self.serialize(sub_wave, relevent_chs=relevent_chs))
            return out_list

        elif issubclass(type(wave), Core_Pulse):
            print("Warning: Using a Core Pulse with no PULSE_TYPE definition.  Could be inefficient")
            return [wave]

        else:
            raise Exception("Received a {}".format(type(wave)))
        
    def compress_zeros(self):
        out = list()
        zero_t = 0
        for wave in self.seq:
            if type(wave) == wave_lib.Zero:
                zero_t += wave.t
            else:
                if zero_t:
                    out.append(wave_lib.Zero(t=zero_t, ch=Channel.no_ch))
                    zero_t = 0
                out.append(wave)
        if zero_t:
            out.append(wave_lib.Zero(t=zero_t, ch=Channel.no_ch))
        self.seq = out

    def re_id_zeros(self):
        zeros_dict = {}
        for i, wave in enumerate(self.seq):
            if type(wave) is wave_lib.Zero:
                if wave.t in zeros_dict:
                    self.seq[i] = zeros_dict[wave.t]
                else:
                    zeros_dict[wave.t] = wave

class Smart_Sim():
    def __init__(self, H_class, H_args=[], sim_chs=[], H_kwargs={}, options=qutip.solver.Options()):
        self.H_class, self.H_args, self.sim_chs, self.H_kwargs, self.options = H_class, H_args, sim_chs, H_kwargs, options
        self.propagators = dict()
    
    def run(self, wave, experimental_rate, sim_rate=None, psi0=qutip.basis(3,1), reuse_propagators=True, verbose=False):
        sim_rate = experimental_rate if sim_rate is None else sim_rate
        relevent_chs = set().union(*[sim_ch.chs for sim_ch in self.sim_chs])
        sim_seq = Sequencer(wave, relevent_chs=relevent_chs)
        
        psi = psi0
        self.states = list()
        self.sims = list()
        self.ts = np.array([])
        next_time = 0
        
        #Build Hamiltonian seq
        Hs = dict()
        for sim_chunk in sim_seq.seq:
            H = self.H_class(*self.H_args, **self.H_kwargs)
            for sim_ch in self.sim_chs:
                #If the sim_ch as some of the same channels as the chunk
                if len(sim_ch.chs.intersection(sim_chunk.chs)):
                    for signal in sim_ch.get_signals(sim_chunk, experiment_rate=experimental_rate):
                        if H.has_interaction(signal.get_interaction_label()):
                            H.add_signal(signal)
            Hs[sim_chunk] = H
            
        #Reset the propagator dict if necessary
        if not reuse_propagators:
            self.propagators = dict()
            
        # Perform the simulation
        for sim_chunk in sim_seq.seq:
            sim_start_time = time.time()
            
            ts = np.linspace(0, sim_chunk.t, int(sim_chunk.t*sim_rate), endpoint=False) + next_time
            
            # For constant wave use the propagator method
            if Hs[sim_chunk].is_constant():
                if sim_chunk in self.propagators:
                    U_t = self.propagators[sim_chunk]
                    print("Reusing U_t")
                else:
                    H = Hs[sim_chunk].get_qutip_descriptor(time_offset=next_time)
                    U_t = qutip.propagator(H, ts, c_op_list=[], options=self.options)
                    if reuse_propagators:
                        self.propagators[sim_chunk] = U_t
                        print("Saved U_t")
                states = U_t*psi
            else:
                sim = Simple_Sim(Hs[sim_chunk], ts, psi0=psi, options=self.options)
                states = sim.output.states
            
            # Update some variables
            self.states.extend(states)
            psi = states[-1]
            self.ts = np.concatenate([self.ts, ts])
            next_time += sim_chunk.t
            
            if verbose:
                print("Simulation of <{}> took {:.4f} s".format(str(sim_chunk), time.time()-sim_start_time))
        return Hs
            
    
    def plot(self,plt=None, method='abs'):
        states = np.array([s.full() for s in self.states])[:,:,0]
        if method == 'abs':
            states = np.abs(states)**2
        elif method == 'real':
            states = np.real(states)
        elif method == 'imag':
            states = np.imag(states)

        cs = ['b', 'g', 'r', 'c', 'y', 'w','m']
        H = self.H_class(*self.H_args, **self.H_kwargs)
        if plt is None:
            plt = pg.plot()
        if not H.labels is None:
            plt.addLegend()
            for i, ys in enumerate(states.T):
                plt.plot(self.ts, ys, pen=cs[i%len(cs)], name=H.labels[i])
        else:
            for i, ys in enumerate(states.T):
                plt.plot(self.ts, ys, pen=cs[i%len(cs)])
        return plt

In [ ]:
%%time
#Not Working
options = qutip.solver.Options()#, max_step=0.1e-9)
iq = Sim_Chs.SG_IQ(f_plus, chs['sg1_i'], chs['sg1_q'])
ssim = Smart_Sim(Single_VV0_Spin, H_args=[PL2['D']], sim_chs=[iq], options=options)
Hs = ssim.run(wave, 10e9, reuse_propagators=True, verbose=True)
ssim.plot()#method='real')

In [ ]:
a = list(Hs.items())
a

In [ ]:
H = a[0][1]
sim_chunk = a[0][0]
arr = np.array(H.signals['Bx'][0].test_ts)

In [ ]:
arr[::1000]

In [ ]:
ts = np.linspace(0, sim_chunk.t, int(sim_chunk.t*1e9), endpoint=False) + 1e-6
sim = Simple_Sim(H, ts, psi0=psi, options=options)
sim.plot(ts)

# Playing with Qutip functions

The cells bellow are simply reminders of some usefull Qutip functions and how to use them.  Everything you need to know about the specific easy_wave implemention is already above

### Building an example problem

In [ ]:
def vv0_H(defect=PL2, B=[0,0,0]):
    #Construct the usual spin matrices
    Si = Sx,Sy,Sz = qutip.jmat(1)
    
    #Construct the Hamiltonian
    H0 = (2*np.pi)*defect['D']*Sz*Sz #Zero-Field splitting
    H = [H0]
    
    #Zeeman-Splitting
    for i, Bi in enumerate(B):
        if Bi:
            H.append([(2*np.pi)*defect['gamma_e']*Si[i], Bi])
    
    return H

psi0 = qutip.basis(3,1)
Bz = 0
w = (PL2['D']+Bz*PL2['gamma_e'])*2*np.pi
T = (2*np.pi)/w
H = vv0_H(B=[lambda t, args: np.sin(w*t),0,lambda t, args: Bz])

In [ ]:
# Defining a plotting function
def plot(output):
    states = np.array([s.full() for s in output.states])[:,:,0]
    states = np.abs(states)**2

    labels = ['+1', ' 0', '-1']
    colors = ['b', 'g', 'r', 'c', 'y', 'w','m', 'b', 'g', 'r', 'c', 'm', 'y', 'w']
    plt = pg.plot()
    plt.addLegend()
    for i, ys in enumerate(states.T):
        plt.plot(ts, ys, pen=colors[i], name=labels[i])
    return plt

### Solving using Master Equation

In [ ]:
%%time
#For time independent hamiltonian
plus = (1/np.sqrt(2))*(qutip.basis(3,0) + qutip.basis(3,1))

# This is super quick!
ts = np.linspace(0,10e-6,1000)
Sx,Sy,Sz = qutip.jmat(1)
H0 = [H[0]]
output = qutip.sesolve(H0, psi0, ts)


In [ ]:
%%time
# For 10us 600 points an lower is too little, so we do 700 points
ts = np.linspace(0,10e-6,700)

# This will take about 10sec to simulate
output = qutip.sesolve(H, psi0, ts)

In [ ]:
plot(output)

### Qutip Simulation using Floquet Formalism

Here we'll attempt to follow http://qutip.org/docs/4.1/guide/dynamics/dynamics-floquet.html to apply Floquet formalism to an isolated spin

In [ ]:
%%time
Si = Sx,Sy,Sz = qutip.jmat(1)
Ts = np.linspace(0, T, 500 + 1)
ts = np.linspace(0,1e-6,10)

# Find the floquet modes and energies
f_modes_0, f_energies = qutip.floquet_modes(H,2*np.pi/w, args={})

# Precompute the mode table
f_modes_table_t = qutip.floquet_modes_table(f_modes_0, f_energies, ts , H, T, args={})

In [ ]:
%%time
# Here we can use as few points as we want
# Eg. start and finish times only
# ts = [0, 10e-6]
# Or we can also do the full time but with less point then before
ts = np.linspace(0,10e-6,100)

# Solve the floquet-markov master euqation
# This will only take ~250ms
output = qutip.fsesolve(H, psi0, ts, e_ops=[], T=T, args={})

In [ ]:
plot(output)

### Qutip Propagator Method

Finally, this is what I had been looking for!

Actually because of the phases of the pulse this is a little tricky...  Not working right now...

In [ ]:
%%time
Sx,Sy,Sz = qutip.jmat(1)
H0 = (2*np.pi)*PL2['D']*Sz*Sz
Hx = (2*np.pi)*PL2['gamma_e']*Sx
w = PL2['D']*2*np.pi
H =[H0, [Hx, lambda t, args: 10*np.sin(w*t)]]

ts = np.linspace(0,1e-8,1000)

U_t = qutip.propagator(H, ts, c_op_list=[])
U0_t = qutip.propagator(H0, ts, c_op_list=[])

In [ ]:
%%time
psi = psi0
states = np.array([])
ts_arr = np.array([])
B_arr = np.array([])
next_time = 0
for i in range(25):
    if i%2:
#         mod_U_t = U_t*np.exp(-1j*Sx)
#         _states = U_t*(np.exp(-(next_time*w)*1j)*Sx*psi)
        _states = U_t*psi
        B_arr = np.concatenate([B_arr, np.sin(w*ts)])
    else:
        _states = (U0_t*psi)
        B_arr = np.concatenate([B_arr, np.zeros(len(ts))])
    
    states = np.concatenate([states, _states])
    ts_arr = np.concatenate([ts_arr, ts + next_time])
    psi = _states[-1]
    next_time = ts_arr[-1]

In [ ]:
# Defining a plotting function
def plot(states):
    states = np.array([s.full() for s in states])[:,:,0]
    states = np.abs(states)**2
    labels = ['+1', ' 0', '-1']
    colors = ['b', 'g', 'r', 'c', 'y', 'w','m', 'b', 'g', 'r', 'c', 'm', 'y', 'w']
    plt = pg.plot()
    plt.addLegend()
    for i, ys in enumerate(states.T):
        plt.plot(ts_arr, ys, pen=colors[i], name=labels[i])
    return plt
ts = np.linspace(0,10e-6,1000)
plt = plot(states)
# plt.plot(ts_arr, B_arr)

In [ ]:
len(ts_arr)